In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats
from scipy.signal import find_peaks, detrend
import pywt

from numpy import arange, array, linspace, loadtxt, log2, logspace, mean, polyfit
from numpy import zeros, pi, sin, cos, arctan2, sqrt, real, imag, conj, tile
from numpy import round, interp, diff, unique, where
from pandas import DataFrame, date_range
import matplotlib.dates as mdates
from matplotlib import pyplot
from scipy.stats import pearsonr, mannwhitneyu, kruskal, norm
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GroupKFold
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,f1_score, recall_score, precision_score,roc_auc_score, roc_curve, auc,accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from collections import defaultdict, Counter


import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D,Concatenate,Input,Dropout, Conv2D, MaxPooling2D, Flatten,Multiply,Attention,Dense,concatenate,Masking,BatchNormalization, Reshape
from keras.models import Sequential
from keras.optimizers import Adam
from keras.applications import DenseNet121
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History
history = History()

import pickle
import joblib
import dill
import json
from collections import defaultdict
from scipy.signal import savgol_filter

import warnings
warnings.filterwarnings('ignore')
import gc


2025-04-01 03:38:02.722403: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-01 03:38:02.733685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 03:38:02.747284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 03:38:02.751425: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 03:38:02.761594: I tensorflow/core/platform/cpu_feature_guar

In [22]:
DEXI_folder = '/home/ma98/Compressed_DEXI_Data/'

T1DEXIP_cgm = pd.read_parquet(os.path.join(DEXI_folder, 'T1DEXI_Dataset.csv'))
# T1DEXIP_cgm['LBDTC'] = pd.to_datetime(T1DEXIP_cgm['LBDTC'])

T1DEXIP_cgm['USUBJID'] = T1DEXIP_cgm['USUBJID'].astype(str) 
#T1DEXIP_cgm['USUBJID'] = T1DEXIP_cgm['USUBJID'].str[2:-1] 
T1DEXIP_cgm['USUBJID'] = T1DEXIP_cgm['USUBJID'].astype(int)
T1DEXIP_cgm['LBDTC'] = pd.to_datetime(T1DEXIP_cgm['LBDTC'])
# T1DEXIP_cgm['LBDTC'] = T1DEXIP_cgm['LBDTC'].dt.strftime('%Y-%m-%d %H:%M:%S')

T1DEXIP_cgm = T1DEXIP_cgm[['USUBJID','LBSTRESN','LBDTC','LBTEST']]
df = T1DEXIP_cgm
df
# # # HDF5
# # df.to_hdf('data.h5', key='df', mode='w', complevel=9, complib='blosc')
# # hdf5_size = os.path.getsize('data.h5')

# # Parquet
# df.to_parquet('data.parquet', compression='snappy')
# parquet_size = os.path.getsize('data.parquet')

# # Feather
# df.to_feather('data.feather')
# feather_size = os.path.getsize('data.feather')

# # CSV with gzip
# df.to_csv('data.csv.gz', index=False, compression='gzip')
# csv_size = os.path.getsize('data.csv.gz')

# # print(f"HDF5 file size: {hdf5_size/1000000} bytes")
# print(f"Parquet file size: {parquet_size/1000000} bytes")
# print(f"Feather file size: {feather_size/1000000} bytes")
# print(f"CSV (gzip) file size: {csv_size/1000000} bytes")
# T1DEXI_CGM_Dataset


,USUBJID,LBSTRESN,LBDTC,LBTEST
0,1,115.0,2020-05-11 00:01:17,Glucose
1,1,109.0,2020-05-11 00:06:17,Glucose
2,1,105.0,2020-05-11 00:11:17,Glucose
3,1,106.0,2020-05-11 00:16:18,Glucose
4,1,110.0,2020-05-11 00:21:18,Glucose
...,...,...,...,...
3785750,988,94.0,2021-04-16 23:41:40,Glucose
3785751,988,95.0,2021-04-16 23:46:40,Glucose
3785752,988,93.0,2021-04-16 23:51:40,Glucose
3785753,988,90.0,2021-04-16 23:56:44,Glucose


In [102]:
DEXI_folder = '/home/ma98/T1DEXIP/'

T1DEXIP_cgm = pd.read_sas(os.path.join(DEXI_folder, 'FAMLPI.xpt'), format="xport")
# T1DEXIP_cgm['LBDTC'] = pd.to_datetime(T1DEXIP_cgm['LBDTC'])

# T1DEXIP_cgm['LBTEST'] = T1DEXIP_cgm['LBTEST'].astype(str) 
# # T1DEXIP_cgm['LBTEST'] = T1DEXIP_cgm['LBTEST'].str[2:-1] 
# T1DEXIP_cgm['USUBJID'] = T1DEXIP_cgm['USUBJID'].astype(int)
# T1DEXIP_cgm['LBDTC'] = pd.to_datetime(T1DEXIP_cgm['LBDTC'], unit='s')
# T1DEXIP_cgm['LBDTC'] = T1DEXIP_cgm['LBDTC'].dt.strftime('%Y-%m-%d %H:%M:%S')

# T1DEXIP_cgm = T1DEXIP_cgm[['USUBJID','LBSTRESN','LBDTC','LBTEST']]

print(np.unique(T1DEXIP_cgm["FAMETHOD"]))
print(np.unique(T1DEXIP_cgm["FACAT"]))
print(np.unique(T1DEXIP_cgm["FASCAT"]))

T1DEXIP_cgm


,STUDYID,DOMAIN,USUBJID,FASEQ,FAGRPID,FATESTCD,FATEST,FAOBJ,FACAT,FASCAT,FAORRES,FAORRESU,FASTRESC,FASTRESN,FASTRESU,FAMETHOD,FAANMETH,FADTC,FPANLTYP
0,b'T1DEXI-P',b'FA',b'100',2.0,6069.0,b'ALCDRKN',b'Number of Alcoholic Drinks',"b'Almond milk, unsweetened'",b'TAKEN',b'ALCOHOLIC DRINKS',b'0.00',b'DRINK',b'0.00',5.397605e-79,b'DRINK',b'RFPM',b'FNDDS 2017-2018',1.944835e+09,b'Photo'
1,b'T1DEXI-P',b'FA',b'100',3.0,6069.0,b'ALCDRKN',b'Number of Alcoholic Drinks',"b'Almond milk, unsweetened'",b'RETURNED',b'ALCOHOLIC DRINKS',b'0.00',b'DRINK',b'0.00',5.397605e-79,b'DRINK',b'RFPM',b'FNDDS 2017-2018',1.944835e+09,b'Photo'
2,b'T1DEXI-P',b'FA',b'100',4.0,6069.0,b'ALCDRKN',b'Number of Alcoholic Drinks',"b'Almond milk, unsweetened'",b'CONSUMED',b'ALCOHOLIC DRINKS',b'0.00',b'DRINK',b'0.00',5.397605e-79,b'DRINK',b'RFPM',b'FNDDS 2017-2018',1.944835e+09,b'Photo'
3,b'T1DEXI-P',b'FA',b'100',5.0,4857.0,b'ALCDRKN',b'Number of Alcoholic Drinks',"b'Apple, raw'",b'TAKEN',b'ALCOHOLIC DRINKS',b'0.00',b'DRINK',b'0.00',5.397605e-79,b'DRINK',b'RFPM',b'FNDDS 2017-2018',1.944648e+09,b'Photo'
4,b'T1DEXI-P',b'FA',b'100',6.0,4857.0,b'ALCDRKN',b'Number of Alcoholic Drinks',"b'Apple, raw'",b'RETURNED',b'ALCOHOLIC DRINKS',b'0.00',b'DRINK',b'0.00',5.397605e-79,b'DRINK',b'RFPM',b'FNDDS 2017-2018',1.944648e+09,b'Photo'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727923,b'T1DEXI-P',b'FA',b'99',31511.0,2209.0,b'TKNAMT',b'Taken Amount',b'Taffy',b'TAKEN',b'TOTAL',b'16.00',b'g',b'16.00',1.600000e+01,b'g',b'RFPM',b'FNDDS 2017-2018',1.949328e+09,b'Photo'
1727924,b'T1DEXI-P',b'FA',b'99',31512.0,2230.0,b'TKNAMT',b'Taken Amount',b'Taffy',b'TAKEN',b'TOTAL',b'15.60',b'g',b'15.60',1.560000e+01,b'g',b'RFPM',b'FNDDS 2017-2018',1.949414e+09,b'Photo'
1727925,b'T1DEXI-P',b'FA',b'99',31513.0,2241.0,b'TKNAMT',b'Taken Amount',b'Taffy',b'TAKEN',b'TOTAL',b'15.60',b'g',b'15.60',1.560000e+01,b'g',b'RFPM',b'FNDDS 2017-2018',1.949739e+09,b'Photo'
1727926,b'T1DEXI-P',b'FA',b'99',31514.0,2247.0,b'TKNAMT',b'Taken Amount',b'Taffy',b'TAKEN',b'TOTAL',b'15.60',b'g',b'15.60',1.560000e+01,b'g',b'RFPM',b'FNDDS 2017-2018',1.949749e+09,b'Photo'


In [119]:
DEXI_folder = '/home/ma98/T1DEXIP/'

T1DEXIP_cgm = pd.read_sas(os.path.join(DEXI_folder, 'FAMLPM.xpt'), format="xport")

T1DEXIP_cgm['FAMETHOD'] = T1DEXIP_cgm['FAMETHOD'].astype(str) 
T1DEXIP_cgm['FACAT'] = T1DEXIP_cgm['FACAT'].astype(str) 
T1DEXIP_cgm['FASCAT'] = T1DEXIP_cgm['FASCAT'].astype(str) 
T1DEXIP_cgm['FATEST'] = T1DEXIP_cgm['FATEST'].astype(str) 

T1DEXIP_cgm['USUBJID'] = T1DEXIP_cgm['USUBJID'].astype(int)
T1DEXIP_cgm = T1DEXIP_cgm[T1DEXIP_cgm['FACAT'].isin(['CONSUMED', 'Consumed'])]

T1DEXIP_cgm = T1DEXIP_cgm[T1DEXIP_cgm['FATEST']=='Dietary Total Carbohydrate']
T1DEXIP_cgm['FADTC'] = pd.to_datetime(T1DEXIP_cgm['FADTC'], unit='s')
T1DEXIP_cgm['FADTC'] = T1DEXIP_cgm['FADTC'].dt.strftime('%Y-%m-%d %H:%M:%S')

T1DEXIP_cgm = T1DEXIP_cgm[['USUBJID','FAMETHOD','FACAT','FASCAT','FATEST','FADTC','FASTRESN']]
df = T1DEXIP_cgm

# # # Parquet
df.to_parquet('pediatricCarbs.parquet', compression='snappy')
parquet_size = os.path.getsize('pediatricCarbs.parquet')

# Feather
df.to_feather('pediatricCarbs.feather')
feather_size = os.path.getsize('pediatricCarbs.feather')

# CSV with gzip
df.to_csv('pediatricCarbs.csv.gz', index=False, compression='gzip')
csv_size = os.path.getsize('pediatricCarbs.csv.gz')

print(f"Parquet file size: {parquet_size/1000000} bytes")
print(f"Feather file size: {feather_size/1000000} bytes")
print(f"CSV (gzip) file size: {csv_size/1000000} bytes")

T1DEXIP_cgm[:40]

Parquet file size: 0.108292 bytes
Feather file size: 0.225522 bytes
CSV (gzip) file size: 0.048944 bytes


,USUBJID,FAMETHOD,FACAT,FASCAT,FATEST,FADTC,FASTRESN
385,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-14 11:21:47,50.00
386,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-15 12:19:35,50.00
387,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-16 11:58:08,50.00
388,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-17 11:52:00,50.00
389,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-18 12:00:30,50.00
390,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-16 20:02:17,100.00
391,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-17 19:43:39,80.00
392,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-18 19:53:07,50.00
393,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-14 14:47:43,50.00
394,100,SELF-REPORT,Consumed,MACRONUTRIENTS,Dietary Total Carbohydrate,2031-08-16 15:08:00,50.00


In [129]:
DEXI_folder = '/home/ma98/T1DEXIP/Compressed/'

T1DEXI_CGM_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'pediatricCGM.parquet')) 
T1DEXI_CGM_Dataset['LBDTC'] = pd.to_datetime(T1DEXI_CGM_Dataset['LBDTC'])

T1DEXI_basal = pd.read_parquet(os.path.join(DEXI_folder, 'pediatricBasal.parquet'))  
T1DEXI_basal['FADTC'] = pd.to_datetime(T1DEXI_basal['FADTC'], format='%Y-%m-%d %H:%M:%S')

FAMLPM_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'pediatricCarbs.parquet')) 
FAMLPM_Dataset['FADTC'] = pd.to_datetime(FAMLPM_Dataset['FADTC'], format='%Y-%m-%d %H:%M:%S')

T1DEXI_age_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'pediatricAge.parquet')) 

T1DEXI_VS_Dataset = pd.read_csv(os.path.join(DEXI_folder, 'pediatricVS.csv.gz'))  
T1DEXI_VS_Dataset['VSDTC'] = pd.to_datetime(T1DEXI_VS_Dataset['VSDTC'], format='%Y-%m-%d %H:%M:%S')

shared_participants = sorted(set(T1DEXI_CGM_Dataset['USUBJID']) & set(T1DEXI_basal['USUBJID'])&
                             set(FAMLPM_Dataset['USUBJID']) & set(T1DEXI_VS_Dataset['USUBJID'])) 

print(len(shared_participants))


256


In [2]:
DEXI_folder = '/home/ma98/T1DEXIP/Compressed/'

T1DEXI_CGM_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'pediatricCGM.parquet')) 
T1DEXI_CGM_Dataset['LBDTC'] = pd.to_datetime(T1DEXI_CGM_Dataset['LBDTC'])

shared_participants = sorted(set(T1DEXI_CGM_Dataset['USUBJID'])) 

print(len(shared_participants))


261


In [3]:
def get_cgm_prediction_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_prediction_stats(general_stats, 'cgm')
    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'early_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 3),
        'late_night': (current_day_cgm.index.hour >= 3) & (current_day_cgm.index.hour < 6),
        'long_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 8),
        'night_morning': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 12),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_prediction_stats(period_stats, f'cgm_{period}')
        time_range_stats.update(period_stats)
    
    time_in_range_stats = {
        'time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'time_in_range_70_180_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'time_below_range_70_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    cgm_prediction_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return cgm_prediction_stats.astype(np.float16)
def get_daily_cgm_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_stats(general_stats, 'daily_cgm')
    mean_crossings = calculate_crossings(current_day_cgm.values[:,0])
    general_stats['daily_cgm_mean_crossings'] = mean_crossings

    # Add new features
    last_cgm_value = current_day_cgm.values[-1, 0]
    general_stats['last_cgm_value'] = last_cgm_value
    general_stats['last_cgm_sq'] = np.sqrt(last_cgm_value)
    
    diff_10_min = last_cgm_value - current_day_cgm.values[-3, 0]
    general_stats['diff_10_min'] = diff_10_min

    diff_20_min = last_cgm_value - current_day_cgm.values[-5, 0]
    general_stats['diff_20_min'] = diff_20_min

    diff_30_min = last_cgm_value - current_day_cgm.values[-7, 0]
    general_stats['diff_30_min'] = diff_30_min

    diff_50_min = last_cgm_value - current_day_cgm.values[-11, 0]
    general_stats['diff_50_min'] = diff_50_min
    
    # Rate of change (slope) in CGM in the last hours
    slope_last_half_hour = (last_cgm_value - current_day_cgm.values[-7, 0]) / last_cgm_value
    general_stats['slope_last_half_hour'] = slope_last_half_hour
    slope_last_hour = (last_cgm_value - current_day_cgm.values[-13, 0]) / last_cgm_value
    general_stats['slope_last_hour'] = slope_last_hour
    slope_last_2_hour = (last_cgm_value - current_day_cgm.values[-25, 0]) / last_cgm_value
    general_stats['slope_last_2_hour'] = slope_last_2_hour
    
    # SD of CGM in the last hours
    std_last_2_hours = np.std(current_day_cgm.values[-25:, 0])
    general_stats['std_last_2_hours'] = std_last_2_hours
    std_last_hour = np.std(current_day_cgm.values[-13:, 0])
    general_stats['std_last_hour'] = std_last_hour
    
    # Sum of all increments in adjacent CGM observations in the last two hours
    increments_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) > 0])
    general_stats['increments_sum_last_2_hours'] = increments_sum_last_2_hours

    # Sum of all decrements in adjacent CGM observations in the last two hours
    decrements_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) < 0])
    general_stats['decrements_sum_last_2_hours'] = decrements_sum_last_2_hours

    # Maximum increase in adjacent CGM observations in the last two hours
    max_increase_last_2_hours = np.max(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_increase_last_2_hours'] = max_increase_last_2_hours

    # Maximum decrease in adjacent CGM observations in the last two hours
    max_decrease_last_2_hours = np.min(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_decrease_last_2_hours'] = max_decrease_last_2_hours

    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24),
        'late_evening': (current_day_cgm.index.hour >= 21) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_stats(period_stats, f'daily_cgm_{period}')
        time_range_stats.update(period_stats)
    
    time_in_range_stats = {
        'daily_time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'daily_time_in_range_70_180_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'daily_time_below_range_70_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'daily_time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'daily_time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'daily_time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'daily_time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'daily_time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'daily_time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'daily_time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    daily_cgm_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return daily_cgm_stats

def get_daily_cgm_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_stats(general_stats, 'daily_cgm')
    mean_crossings = calculate_crossings(current_day_cgm.values[:,0])
    general_stats['daily_cgm_mean_crossings'] = mean_crossings

    # Add new features
    last_cgm_value = current_day_cgm.values[-1, 0]
    general_stats['last_cgm_value'] = last_cgm_value #11:55
    general_stats['last_cgm_sq'] = np.sqrt(last_cgm_value)
    
    general_stats['diff_10_min'] = last_cgm_value - current_day_cgm.values[-3, 0] #11:45
    general_stats['diff_20_min'] = last_cgm_value - current_day_cgm.values[-5, 0] #11:35
    general_stats['diff_30_min'] = last_cgm_value - current_day_cgm.values[-7, 0] #11:25
    general_stats['diff_40_min'] = last_cgm_value - current_day_cgm.values[-9, 0] #11:15
    general_stats['diff_50_min'] = last_cgm_value - current_day_cgm.values[-11, 0] #11:05
    
    # Rate of change (slope) in CGM in the last hours
    general_stats['slope_last_half_hour'] = (last_cgm_value - current_day_cgm.values[-6, 0]) / last_cgm_value #11:30
    general_stats['slope_last_hour'] = (last_cgm_value - current_day_cgm.values[-13, 0]) / last_cgm_value #10:55
    general_stats['slope_last_hour_half'] = (last_cgm_value - current_day_cgm.values[-19, 0]) / last_cgm_value #10:25
    general_stats['slope_last_2_hour'] = (last_cgm_value - current_day_cgm.values[-25, 0]) / last_cgm_value #9:55
    
    # SD of CGM in the last hours
    general_stats['std_last_2_hours'] = np.std(current_day_cgm.values[-25:, 0]) #9:55
    general_stats['std_last_hour'] = np.std(current_day_cgm.values[-13:, 0]) #10:55
    
    # Sum of all increments in adjacent CGM observations in the last two hours
    increments_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) > 0])
    general_stats['increments_sum_last_2_hours'] = increments_sum_last_2_hours

    # Sum of all decrements in adjacent CGM observations in the last two hours
    decrements_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) < 0])
    general_stats['decrements_sum_last_2_hours'] = decrements_sum_last_2_hours

    # Maximum increase in adjacent CGM observations in the last two hours
    max_increase_last_2_hours = np.max(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_increase_last_2_hours'] = max_increase_last_2_hours

    # Maximum decrease in adjacent CGM observations in the last two hours
    max_decrease_last_2_hours = np.min(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_decrease_last_2_hours'] = max_decrease_last_2_hours
    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24),
        'late_evening': (current_day_cgm.index.hour >= 21) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_stats(period_stats, f'daily_cgm_{period}')
        time_range_stats.update(period_stats)

    time_in_range_stats = {
        'daily_time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'daily_time_in_range_70_180_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'daily_time_below_range_70_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'daily_time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'daily_time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'daily_time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'daily_time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'daily_time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'daily_time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'daily_time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    daily_cgm_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return daily_cgm_stats.astype(np.float16)

def daily_cgm_labels(current_day_cgm):
    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'early_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 3),
        'late_night': (current_day_cgm.index.hour >= 3) & (current_day_cgm.index.hour < 6),
        'long_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 8),
        'night_morning': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 12),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24)
    }
    labels = {
        'hyper_day': return_label(current_day_cgm.values[:,0], 180),
        'hypo_day': return_label(current_day_cgm.values[:,0], 70)
    }
    for period, mask in masks.items():
        labels[f'hyper_{period}'] = return_label(current_day_cgm.values[:,0], 180, mask)
        labels[f'hypo_{period}'] = return_label(current_day_cgm.values[:,0], 70, mask)
    
    daily_cgm_labels = pd.DataFrame({**labels}, index=[0])
    return daily_cgm_labels
# --- Calculate Time in Range for Different Ranges and Periods ---
def calculate_time_in_range(glucose_data, range_lower, range_upper, period_mask=None):

    if period_mask is not None:
        glucose_data = glucose_data[period_mask]
    in_range = (glucose_data <= range_upper) & (glucose_data >= range_lower)
    return 100 * in_range.sum() / len(glucose_data) if len(glucose_data) > 0 else 0
def return_label(glucose_data, range, period_mask=None):
    if period_mask is not None:
        glucose_data = glucose_data[period_mask]
    if range==180:
        return np.sum(glucose_data>range)>2
    if range==70:
        return np.sum(glucose_data<range)>2
def calculate_sleep_statistics(daily_sleep_data):
    participant_total_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Total Sleep Time"]
    participant_deep_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Deep NREM Duration"]
    participant_light_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Light NREM Duration"]
    participant_NREM_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "NREM Duration"]
    participant_REM_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "REM Duration"]
    participant_efficiency = participant_sleep_data[participant_sleep_data['NVTEST'] == "Sleep Efficiency"]
    participant_awakenings = participant_sleep_data[participant_sleep_data['NVTEST'] == "Number of Awakenings"]
    participant_latency = participant_sleep_data[participant_sleep_data['NVTEST'] == "Sleep Onset Latency"]

    avg_total_sleep = (participant_total_sleep['NVORRES']).mean()
    std_total_sleep = (participant_total_sleep['NVORRES']).std()
    avg_deep_sleep = (participant_deep_sleep['NVORRES']).mean()
    std_deep_sleep = (participant_deep_sleep['NVORRES']).std()
    avg_light_sleep = (participant_light_sleep['NVORRES']).mean()
    std_light_sleep = (participant_light_sleep['NVORRES']).std()
    avg_NREM_sleep = (participant_NREM_sleep['NVORRES']).mean()
    std_NREM_sleep = (participant_NREM_sleep['NVORRES']).std()
    avg_REM_sleep = (participant_REM_sleep['NVORRES']).mean()
    std_REM_sleep = (participant_REM_sleep['NVORRES']).std()
    avg_awakenings = (participant_awakenings['NVORRES']*100000.).mean()
    std_awakenings = (participant_awakenings['NVORRES']*100000. ).std()
    avg_latency = (participant_latency['NVORRES']).mean()
    std_latency = (participant_latency['NVORRES']).std()
    avg_efficiency = (participant_efficiency['NVORRES']*100000.).mean()
    std_efficiency = (participant_efficiency['NVORRES']*100000.).std()
    # Calculate avg deviation from midnight
    bedtime_deviations_midnight = []
    wakeup_deviations_midnight = []
    for dt in participant_total_sleep['NVDTC']:
        sleep_time = 60*dt.hour + (dt.minute)
        
        if sleep_time >720:
            sleep_time -= 1440
        bedtime_deviations_midnight.append(sleep_time)
    
    for dt in participant_total_sleep['NVENDTC']:
        wakeup_time = 60*dt.hour + (dt.minute)
        wakeup_deviations_midnight.append(wakeup_time)
  
    average_bedtime_midnight = np.mean(bedtime_deviations_midnight)
    average_wakeup_midnight = np.mean(wakeup_deviations_midnight)
    std_bedtime_midnight = np.std(bedtime_deviations_midnight)
    std_wakeup_midnight =  np.std(wakeup_deviations_midnight)
   #     # Convert average deviation back to time format
    average_bedtime = pd.to_datetime('00:00:00') + pd.to_timedelta(average_bedtime_midnight, unit='m')
    average_wakeup = pd.to_datetime('00:00:00') + pd.to_timedelta(average_wakeup_midnight, unit='m')
       
    # Calculate bedtime deviations from average & variance & Consistency Score (inverse of variance)
    participant_sleep_data['bedtime_from_avg'] = ((60*participant_sleep_data['NVDTC'].dt.hour + participant_sleep_data['NVDTC'].dt.minute) - average_bedtime_midnight)
    participant_sleep_data.loc[participant_sleep_data['bedtime_from_avg'] > 720, 'bedtime_from_avg'] -= 1440

    participant_sleep_data['wakeup_from_avg'] = ((60*participant_sleep_data['NVENDTC'].dt.hour + participant_sleep_data['NVENDTC'].dt.minute) - average_wakeup_midnight)
    bedtime_std = np.round(participant_sleep_data['bedtime_from_avg'].std(),3)
    wakeup_std = np.round(participant_sleep_data['wakeup_from_avg'].std(),3)
    bedtime_var = np.round(participant_sleep_data['bedtime_from_avg'].var(),3)
    wakeup_var = np.round(participant_sleep_data['wakeup_from_avg'].var(),3)
    sleep_stats = pd.DataFrame({
        'avg_bedtime_midnight': [average_bedtime_midnight.round(3)],
        'avg_bedtime': [average_bedtime],
        'bedtime_consistency_score': [(100. / bedtime_var).round(3)],
        'bedtime_std': [bedtime_std],
        'bedtime_var': [bedtime_var],
        'avg_wakeup_midnight': [average_wakeup_midnight.round(3)],
        'avg_wakeup': [average_wakeup],
        'wakeup_consistency_score': [(100. / wakeup_var).round(3)],
        'wakeup_std': [wakeup_std],
        'wakeup_var': [wakeup_var],
        'avg_deep_sleep': [round(avg_deep_sleep, 3)],
        'std_deep_sleep': [round(std_deep_sleep, 3)],
        'avg_total_sleep': [round(avg_total_sleep, 3)],
        'std_total_sleep': [round(std_total_sleep, 3)],
        'avg_light_sleep': [round(avg_light_sleep, 3)],
        'std_light_sleep': [round(std_light_sleep, 3)],
        'avg_NREM_sleep': [round(avg_NREM_sleep, 3)],
        'std_NREM_sleep': [round(std_NREM_sleep, 3)],
        'avg_REM_sleep': [round(avg_REM_sleep, 3)],
        'std_REM_sleep': [round(std_REM_sleep, 3)],
        'avg_awakenings': [round(avg_awakenings, 3)],
        'std_awakenings': [round(std_awakenings, 3)],
        'avg_latency': [round(avg_latency, 3)],
        'std_latency': [round(std_latency, 3)],
        'avg_efficiency': [round(avg_efficiency, 3)],
        'std_efficiency': [round(std_efficiency, 3)]
    })
    return participant_sleep_data, sleep_stats.astype(np.float16)
def calculate_stats(data, period_mask=None):
    if period_mask is not None:
        data = data[period_mask]
        data = data.values[:,0]
    data = data[~np.isnan(data)]
    if len(data) == 0:
        return pd.DataFrame({
            'avg': [np.nan],
            'std': [np.nan],
            'min': [np.nan],
            'n5': [np.nan],
            'n25': [np.nan],
            'median': [np.nan],
            'n75': [np.nan],
            'n95': [np.nan],
            'max': [np.nan],
            'var': [np.nan],
            'std_score': [np.nan],
            'var_score': [np.nan],
            'consistency_score': [np.nan],
            'entropy': [np.nan]
        })
    max_val = round(np.nanmax(data), 3)
    min_val = round(np.nanmin(data), 3)
    avg_val = round(np.nanmean(data), 3)
    median = round(np.nanpercentile(data, 50),3)
    
    if len(data) > 1:
        n5 = round(np.nanpercentile(data, 5), 3)
        n25 = round(np.nanpercentile(data, 25), 3)
        n75 = round(np.nanpercentile(data, 75), 3)
        n95 = round(np.nanpercentile(data, 95), 3)
        std_val = round(np.nanstd(data), 3)
        var_val = round(np.nanvar(data), 3)
        std_score_val = round(100 * np.nanstd(data) / np.nanmean(data), 3)
        var_score_val = round(np.nanvar(data) / np.nanmean(data), 3)
        consistency_score_val = round(100. / np.nanvar(data), 3) if np.nanvar(data) != 0 else np.nan
        entropy = calculate_entropy(data)
    else:
        n5 = np.nan
        n25 = np.nan
        n75 = np.nan
        n95 = np.nan
        std_val = np.nan
        var_val = np.nan
        std_score_val = np.nan
        var_score_val = np.nan
        consistency_score_val = np.nan
        entropy = np.nan
    data_stats = pd.DataFrame({
        'avg': [avg_val],
        'std': [std_val],
        'min': [min_val],
        'n5': [n5],
        'n25': [n25],
        'median': [median],
        'n75': [n75],
        'n95': [n95],
        'max': [max_val],
        'var': [var_val],
        'std_score': [std_score_val],
        'var_score': [var_score_val],
        'consistency_score': [consistency_score_val],
        'entropy': [entropy]
    })
    return data_stats.astype(np.float16)
# Helper function to extract values from the DataFrame
def extract_stats(stats_df, prefix):
    return {
        f'{prefix}_avg': float(stats_df['avg'].values[0]),
        f'{prefix}_std': float(stats_df['std'].values[0]),
        f'{prefix}_min': float(stats_df['min'].values[0]),
        f'{prefix}_n5': float(stats_df['n5'].values[0]),
        f'{prefix}_n25': float(stats_df['n25'].values[0]),
        f'{prefix}_median': float(stats_df['median'].values[0]),
        f'{prefix}_n75': float(stats_df['n75'].values[0]),
        f'{prefix}_n95': float(stats_df['n95'].values[0]),
        f'{prefix}_max': float(stats_df['max'].values[0]),
        f'{prefix}_var': float(stats_df['var'].values[0]),
        f'{prefix}_std_score': float(stats_df['std_score'].values[0]),
        f'{prefix}_var_score': float(stats_df['var_score'].values[0]),
        f'{prefix}_consistency_score': float(stats_df['consistency_score'].values[0]),
        f'{prefix}_entropy': float(stats_df['entropy'].values[0]),
    }
def extract_prediction_stats(stats_df, prefix):
    return {
        f'{prefix}_max': float(stats_df['max'].values[0]),
        f'{prefix}_median': float(stats_df['median'].values[0]),
        f'{prefix}_avg': float(stats_df['avg'].values[0]),
        f'{prefix}_std_score': float(stats_df['std'].values[0])
    }
def extract_reduced_stats(stats_df, prefix):
    return {
        f'{prefix}_median': float(stats_df['median'].values[0]),
        f'{prefix}_avg': float(stats_df['avg'].values[0]),
        f'{prefix}_entropy': float(stats_df['entropy'].values[0]),
        f'{prefix}_std_score': float(stats_df['std'].values[0]),
        f'{prefix}_var_score': float(stats_df['var'].values[0])
    }
def calculate_sum(carbs_data, period_mask=None):
    if period_mask is not None:
        carbs_data = carbs_data[period_mask]
    return np.sum(carbs_data['corrected_value']) if not carbs_data['corrected_value'].empty else 0
def corrected_daily_carbs_stats(daily_carb_data):
    overall_stats = calculate_stats(daily_carb_data['corrected_value'].values)
    overall_stats = extract_prediction_stats(overall_stats, 'carbs')
    
    if len(daily_carb_data) > 0:
        # Calculate carb on board at midnight considering only intake after 7 PM
        start_time = pd.Timestamp(daily_carb_data['FADTC'].dt.date.iloc[0]) + pd.Timedelta(hours=19)  # 7 PM
        midnight = pd.Timestamp(daily_carb_data['FADTC'].dt.date.iloc[0]) + pd.Timedelta(days=1)  # 12 AM next day
        carb_on_board = 0.0
        for _, row in daily_carb_data[daily_carb_data['FADTC'] >= start_time].iterrows():
            carb_amount = row['corrected_value']
            intake_time = row['FADTC']
            absorption_start_time = intake_time + pd.Timedelta(minutes=15)

            if midnight > absorption_start_time:
                absorbed_duration = min((midnight - absorption_start_time).total_seconds() / 60, carb_amount / 0.5)
                absorbed_carbs = absorbed_duration * 0.5
                remaining_carbs = max(carb_amount - absorbed_carbs, 0)
                carb_on_board += remaining_carbs
    else:
        carb_on_board = np.nan
    overall_stats['carb_on_board_midnight'] = carb_on_board
    masks = {
        'night': (daily_carb_data['FADTC'].dt.hour >= 0) & (daily_carb_data['FADTC'].dt.hour < 6),
        'morning': (daily_carb_data['FADTC'].dt.hour >= 6) & (daily_carb_data['FADTC'].dt.hour < 12),
        'afternoon': (daily_carb_data['FADTC'].dt.hour >= 12) & (daily_carb_data['FADTC'].dt.hour < 18),
        'evening': (daily_carb_data['FADTC'].dt.hour >= 18) & (daily_carb_data['FADTC'].dt.hour < 24),
        'late_evening': (daily_carb_data['FADTC'].dt.hour >= 21) & (daily_carb_data['FADTC'].dt.hour < 24)
    } 
    time_range_stats = {}
    if len(daily_carb_data) > 0:
        time_range_stats['carbs_day'] = calculate_sum(daily_carb_data)
    else:
        time_range_stats['carbs_day'] = np.nan
        
    for period, mask in masks.items():
        if len(daily_carb_data) > 0:
            time_range_stats[f'carbs_{period}'] = calculate_sum(daily_carb_data, mask)
        else:
            time_range_stats[f'carbs_{period}'] = np.nan

    carbs_stats = pd.DataFrame({**overall_stats, **time_range_stats}, index=[0])
    return carbs_stats.astype(np.float16)
def total_stats(data, times, label_data='bolus'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24),
        'late_evening': (daily_data.index.hour >= 22) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    total_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return total_stats.astype(np.float16)
def daily_stats_hr(data, times, label_data='hr'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24),
        'late_evening': (daily_data.index.hour >= 21) & (daily_data.index.hour < 24),
        'before_sleep': (daily_data.index.hour >= 23) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats.astype(np.float16)
    
def daily_stats(data, times, label_data='basal'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24),
        'late_evening': (daily_data.index.hour >= 22) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats.astype(np.float16)
def daily_stats_basal(data, times, label_data='basal'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24),
        'late_evening': (daily_data.index.hour >= 22) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats.astype(np.float16)
def daily_stats_bolus(data, times, label_data='bolus'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats.astype(np.float16)
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=stats.entropy(probabilities)
    return entropy.round(3)

def calculate_crossings(list_values):
    mean_crossing_indices = np.nonzero(np.diff(list_values > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return no_mean_crossings

def resample_avg(data, times, interval='5T'):
    df_data = pd.DataFrame({'datetime': times, 'value': data})
    df_data.set_index('datetime', inplace=True)
    data_resampled = df_data.copy()
    data_resampled.index = data_resampled.index.floor(interval)
    data_resampled = data_resampled.groupby(data_resampled.index).mean()
    data_resampled = data_resampled.resample(interval).asfreq()
    return data_resampled

def smooth_signal(df, window_size=17, treshold=10, order=None):
    data = df['value']

    if order is not None:
        # Temporarily fill NaNs with interpolation to avoid issues with savgol_filter
        data_filled = data.interpolate(method='linear')
        # Apply savgol_filter only to non-NaN values
        smoothed_filled = savgol_filter(data_filled.fillna(0), window_length=window_size-8, polyorder=order)
        # Reintroduce NaNs where they were originally
        smoothed_values = pd.Series(smoothed_filled, index=data.index)
        smoothed_values[np.isnan(data)] = np.nan
    else:
        smoothed_values = data
        
    half_window = window_size // 2
    # Generate a Gaussian (normal) distribution for weights
    # The mean is at the center of the window, and the standard deviation determines the spread
    weights = norm.pdf(np.arange(-half_window, half_window + 1), 0, 2)
    # Normalize weights so that they sum to 1
    weights /= weights.sum()
    data_smooth = pd.Series(smoothed_values, index=df.index)
    rolling_avg = data_smooth.rolling(window=window_size, center=True).apply(lambda x: np.dot(x, weights), raw=True)

    #rolling_avg = data.rolling(window=window_size, center=True).mean()
    fluctuations = data - rolling_avg
    smoothed_values = rolling_avg + fluctuations.where(fluctuations.abs() > treshold, 0)
    
    weights = norm.pdf(np.arange(-3, 3 + 1), 0, 2)
    weights /= weights.sum()
    rolling_avg = smoothed_values.rolling(window=7, center=True).apply(lambda x: np.dot(x, weights), raw=True)
    fluctuations = smoothed_values - rolling_avg
    smoothed_values = rolling_avg + fluctuations.where(fluctuations.abs() > treshold, 0)

    data_filled = smoothed_values.interpolate(method='linear')
    # Apply savgol_filter only to non-NaN values
    smoothed_filled = savgol_filter(data_filled.fillna(0), window_length=window_size-10, polyorder=order+1)
    # Reintroduce NaNs where they were originally
    smoothed_values = pd.Series(smoothed_filled, index=data.index)
    smoothed_values[np.isnan(data)] = np.nan
        
    smoothed_df = pd.DataFrame({'datetime': df.index, 'value': smoothed_values})
    smoothed_df.set_index('datetime', inplace=True)
    return smoothed_df
    
def compute_cwt(data, scales, dt=1./12, normalize = True, waveletname = 'mexh', label_data = 'CGM ' ):
    times = data.index
    N = len(data)
    if waveletname in ['cmor', 'shan']:
        waveletname += '1.5-1'
        
    [coefficients_mex, frequencies_mex] = pywt.cwt(data.values.squeeze(), scales, waveletname, dt)
    [coefficients_morl, frequencies_morl] = pywt.cwt(data.values.squeeze(), 4*scales, 'morl', dt)
    # period_mex = 1. / frequencies_mex
    # period_morl = 1. / frequencies_morl
    # fft = np.fft.fft(data)
    # fftfreqs = np.fft.fftfreq(N, dt)
    scaleMatrix = np.ones([1, N]) * scales[:, None]

    power_mex =  (abs(coefficients_mex)) ** 2 / scaleMatrix
    power_morl =  (abs(coefficients_morl)) ** 2 / scaleMatrix
    # Other metrics to analyze later
    # signed_log_power_mex = np.sign(coefficients_mex) * np.log2(power_mex)
    # signed_log_power_morl = np.sign(coefficients_morl) * np.log2(power_morl)
    # Replace -inf with the minimum finite value
    # signed_log_power_mex = np.where(np.isfinite(signed_log_power_mex), signed_log_power_mex, np.min(signed_log_power_mex[np.isfinite(signed_log_power_mex)])/10)
    # signed_log_power_morl = np.where(np.isfinite(signed_log_power_morl), signed_log_power_morl, np.min(signed_log_power_morl[np.isfinite(signed_log_power_morl)])/10)
    return (
        coefficients_mex,
        power_mex,
        # signed_log_power_mex.astype(np.float32),
        coefficients_morl,
        power_morl
        # signed_log_power_morl.astype(np.float32)
        # fft.astype(np.float32),
        # fftfreqs.astype(np.float32)
    )
def remove_nan_rows(X_daily, X_individual, y_label):
    # Find rows with NaNs in either feature vector
    nan_mask_daily = np.any(np.isnan(X_daily), axis=(1, 2))
    nan_mask_individual = np.any(np.isnan(X_individual), axis=(1, 2))
    
    nan_mask_combined = nan_mask_daily | nan_mask_individual    
    # Keep only rows without NaNs
    X_daily_clean = X_daily[~nan_mask_combined]
    X_individual_clean = X_individual[~nan_mask_combined]
    y_label_clean = y_label[~nan_mask_combined]
    return X_daily_clean, X_individual_clean, y_label_clean


In [137]:
gc.collect()
data_dict = defaultdict(lambda: defaultdict(dict))
for participant_id in shared_participants: 
    participant_insulin_data = T1DEXI_basal[T1DEXI_basal['USUBJID'] == participant_id].reset_index()     
    participant_cgm_data = T1DEXI_CGM_Dataset[T1DEXI_CGM_Dataset['USUBJID'] == participant_id].reset_index()
    participant_hr_data = T1DEXI_VS_Dataset[T1DEXI_VS_Dataset['USUBJID'] == participant_id].reset_index()
    # Less than two days
    if len(participant_cgm_data)<550:
        continue
        
    participant_carbs_self = FAMLPM_Dataset[FAMLPM_Dataset['USUBJID']==participant_id][FAMLPM_Dataset['FAMETHOD']=='SELF-REPORT'].reset_index()
    participant_carbs_RFPM = FAMLPM_Dataset[FAMLPM_Dataset['USUBJID']==participant_id][FAMLPM_Dataset['FAMETHOD']=='RFPM'].reset_index()

    merged_carbs = pd.merge(participant_carbs_self[['FASTRESN','FADTC']], participant_carbs_RFPM[['FASTRESN','FADTC']], on='FADTC', how='left', suffixes=('_self', '_RFPM'))
    merged_carbs['corrected_value'] = (merged_carbs['FASTRESN_self'] + merged_carbs['FASTRESN_RFPM']) / 2
    participant_carbs_self['corrected_value'] = merged_carbs['corrected_value'].fillna(participant_carbs_self['FASTRESN'])        
    participant_carbs_self = participant_carbs_self.dropna(subset=['corrected_value']).reset_index()
    if participant_hr_data.empty:
        continue
    a1c_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Hemoglobin A1c"]
    if a1c_data.empty:
        continue
    else:
        a1c = a1c_data['LBSTRESN'].iloc[0]
    height_data = participant_hr_data[participant_hr_data['VSTESTCD'] == "HEIGHT"]
    height = height_data['VSSTRESN'].iloc[0]
    weight_data = participant_hr_data[participant_hr_data['VSTESTCD'] == "WEIGHT"]
    weight = weight_data['VSSTRESN'].iloc[0]
    print("\nparticipant id: ", participant_id)

    # --- Separate data types ---
    participant_basal_data = participant_insulin_data[participant_insulin_data['FACAT'] == "BASAL"][participant_insulin_data['FATESTCD'] == "INSBASAL"].dropna(subset=['FASTRESN']).reset_index() #BASFLRT  #INSBASAL
    participant_cgm_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Glucose"].dropna(subset=['LBSTRESN']).reset_index()
    participant_hr_data = participant_hr_data[participant_hr_data['VSTESTCD']=='HRM'].dropna(subset=['VSSTRESN']).reset_index()

    # --- Align data to 5-minute intervals ---    
    aligned_cgm = resample_avg(participant_cgm_data['LBSTRESN'], participant_cgm_data['LBDTC'], interval='5T')
    # --- Interpolate max 3 hour gaps ---    
    method = 'linear'  
    interpolated_cgm = aligned_cgm.interpolate(method=method, order=2,limit_direction='both')
    smoothed_cgm = smooth_signal(interpolated_cgm, window_size=17, treshold=10, order=2)
    interpolated_cgm = smoothed_cgm.interpolate(method=method, order=2,limit_direction='both')

    # Total Participant Stats
    total_basal_stats = total_stats(participant_basal_data['FASTRESN'], participant_basal_data['FADTC'], 'basal')
    total_hr_stats = total_stats(participant_hr_data['VSSTRESN'], participant_hr_data['VSDTC'], 'hr')
    total_carbs_stats = total_stats(participant_carbs_self['corrected_value'], participant_carbs_self['FADTC'], 'carb')

# --- Compute & Plot overall CWT ---    
    dt_cgm = 1./12 
    s0 = 1./2  #starting period is 15 minutes
    dj = 1/8  # Resolusion
    J = int(12 / dj)
    scales = s0 * np.arange(1,576,2)
    coefficients_mex,power_mex,coefficients_morl,power_morl = compute_cwt(interpolated_cgm, scales, dt=dt_cgm , normalize = False, waveletname = 'mexh', label_data = "CGM ID#" +str(participant_id) )
 
    # --- Find overlapping dates ---
    unique_dates = sorted(set(interpolated_cgm.index.date))
    unique_dates = unique_dates[:-1] # Skip the last day
    for current_date in unique_dates:
        # --- Get current day data (for feature extraction) ---
        current_day_mask_cgm = interpolated_cgm.index.date == current_date
        current_day_indices_cgm = np.where(current_day_mask_cgm)[0]
        
        current_day_basal = participant_basal_data[participant_basal_data['FADTC'].dt.date == current_date]
        current_day_mask_cgm = current_day_mask_cgm.astype(bool)
        current_day_cgm = interpolated_cgm[current_day_mask_cgm]
        current_day_cgm_original = aligned_cgm[aligned_cgm.index.date == current_date]
        current_day_hr = participant_hr_data[participant_hr_data['VSDTC'].dt.date == current_date]
        current_day_carbs = participant_carbs_self[participant_carbs_self['FADTC'].dt.date == current_date]
        if len(current_day_cgm_original)<288 or current_day_cgm_original.isna().sum()[0]>60 or current_day_cgm_original[-48:].isna().sum()[0]>5:
            continue

        next_date = current_date + pd.Timedelta(days=1) # Get one hour before 
        next_day_cgm_original = aligned_cgm[aligned_cgm.index.date == next_date]
        next_night = current_day_cgm.index[-1] + pd.Timedelta(hours=12) 
        next_half_day = next_day_cgm_original[next_day_cgm_original.index <= next_night] #More than 2hour missing in the next 12 hours
        if next_half_day.isna().sum()[0]>36:
            continue

        current_day_carbs_stats = corrected_daily_carbs_stats(current_day_carbs)   
        current_day_cgm_stats_original = get_daily_cgm_stats(current_day_cgm_original)  
        current_day_basal_stats = daily_stats_basal(current_day_basal['FASTRESN'], current_day_basal['FADTC'], 'basal_daily')
        current_day_hr_stats = daily_stats_hr(current_day_hr['VSSTRESN'], current_day_hr['VSDTC'], 'hr_daily')
        next_day_cgm_stats_original = get_cgm_prediction_stats(next_day_cgm_original)
        next_day_cgm_labels_original = daily_cgm_labels(next_day_cgm_original)
        
        matrix_shape0 = len(scales)
        matrix_shape1 = len(current_day_cgm)
        # daily_power_mex = power_mex[:, current_day_indices_cgm]
        # daily_coeffs_mex = coefficients_mex[:, current_day_indices_cgm]
        # daily_power_morl = power_morl[:, current_day_indices_cgm]
        # daily_coeffs_morl = coefficients_morl[:, current_day_indices_cgm]
        
        full_matrix1 = np.ndarray(shape=(matrix_shape0, matrix_shape1,3))
        full_matrix1[:, :, 0] = coefficients_mex[:, current_day_indices_cgm]
        full_matrix1[:, :, 1] = power_mex[:, current_day_indices_cgm]
        full_matrix1[:, :, 2] = power_morl[:, current_day_indices_cgm]

        
        # Store data in the dictionary
        data_dict[participant_id][current_date] = {
            'len_original_cgm': len(current_day_cgm_original),
            'full_matrix_cgm_power': full_matrix1,
            
            'next_day_cgm_stats_original':next_day_cgm_stats_original,
            'next_day_cgm_labels_original':next_day_cgm_labels_original,
            'CGM_stats_daily_original': current_day_cgm_stats_original,
            
            'Basal_stats_daily': current_day_basal_stats,
            'Carbs_stats_daily': current_day_carbs_stats,
            'HR_stats_daily': current_day_hr_stats,
            
            'A1c': a1c,
            'Weight': weight,
            'Height': height,
            'Basal_stats_participant': total_basal_stats,
            'Carb_stats_participant': total_carbs_stats,
            'HR_stats_participant': total_hr_stats
        }
       


participant id:  6

participant id:  7

participant id:  8

participant id:  9

participant id:  11

participant id:  13

participant id:  16

participant id:  18

participant id:  19

participant id:  23

participant id:  24

participant id:  25

participant id:  26

participant id:  29

participant id:  33

participant id:  34

participant id:  35

participant id:  36

participant id:  37

participant id:  40

participant id:  44

participant id:  45

participant id:  46

participant id:  47

participant id:  55

participant id:  57

participant id:  58

participant id:  65

participant id:  66

participant id:  67

participant id:  68

participant id:  69

participant id:  70

participant id:  71

participant id:  73

participant id:  75

participant id:  76

participant id:  77

participant id:  83

participant id:  84

participant id:  86

participant id:  88

participant id:  91

participant id:  92

participant id:  99

participant id:  100

participant id:  101

participant id

In [15]:
with open('/home/ma98/data_generated/Pediatric/FullFeatures/full_pediatric_cgm_mex_morl_smooth.dill', 'rb') as f:
    filtered_data_dict = dill.load(f)

# Prepare the groups based on participant IDs
participant_ids = []
X_features_individual_cgm, X_features_daily_cgm = [], []
X_features_individual, X_features_daily = [], []
X_matrices_power = [] #, X_matrices_coeffs
y_labels = { 
    "hypo_early_night": [], "hypo_night": [], "hypo_long_night": [],
    "hypo_night_morning": [], "hyper_day": [], "hypo_late_night": [],
    "hyper_night": [], "hyper_early_night": [], "hypo_morning": [],
    "hypo_day": []
}

for participant_id, days_data in filtered_data_dict.items():
    participant_ids.extend([participant_id] * len(days_data))
    for day_data in days_data:
        X_matrices_power.append(day_data['full_matrix_cgm_power'])
        # X_matrices_coeffs.append(day_data['full_matrix_cgm_coeffs'])
        X_features_daily_cgm.append(day_data['CGM_stats_daily_original'])
        X_features_individual_cgm.append(np.concatenate([
            [[day_data['A1c']]],
            day_data['CGM_stats_participant']], axis=1))
        X_features_daily.append(np.concatenate([
            day_data['CGM_stats_daily_original'],
            day_data['Basal_stats_daily'],
            day_data['Carbs_stats_daily'],
            day_data['HR_stats_daily']], axis=1))
        X_features_individual.append(np.concatenate([
            [[day_data['A1c']]],
            [[day_data['Weight']]],
            [[day_data['Height']]],
            day_data['Basal_stats_participant'],
            day_data['HR_stats_participant']], axis=1))
    
        # Store labels in the dictionary
        for label in y_labels.keys():
            y_labels[label].append(day_data['next_day_cgm_labels_original'][label])
    
participant_ids = np.array(participant_ids)
X_matrices_power = np.array(X_matrices_power)
# X_matrices_coeffs = np.array(X_matrices_coeffs)
X_features_daily_cgm = np.array(X_features_daily_cgm)
X_features_individual_cgm = np.array(X_features_individual_cgm)
X_features_daily = np.array(X_features_daily)
X_features_individual = np.array(X_features_individual)

# Convert labels to NumPy arrays
for key in y_labels:
    y_labels[key] = np.array(y_labels[key], dtype=np.float32)

# Normalize the individual features
def standard_scale(data):
    scaler = StandardScaler()
    if np.isinf(data).any():
        print("There are infinite values in the data.")
        
        # Replace infinite values with NaN (or a large number)
        data[np.isinf(data)] = np.nan
    return scaler.fit_transform(data.reshape(-1, data.shape[-1]))
    
# Apply vectorized scaling
X_features_individual_cgm_scaled = standard_scale(X_features_individual_cgm)
X_features_daily_cgm_scaled = standard_scale(X_features_daily_cgm)
X_features_individual_scaled = standard_scale(X_features_individual)
X_features_daily_scaled = standard_scale(X_features_daily)

def check_nans(arr, name):
    print(f"{name}: {np.isnan(arr).sum()}  NaNs with shape: {arr.shape}")

print('NaNs in the Data ...')
check_nans(X_matrices_power, "X_matrices_power")
# check_nans(X_matrices_coeffs, "X_matrices_coeffs")
check_nans(X_features_daily_cgm_scaled, "X_features_daily_cgm")
check_nans(X_features_individual_cgm_scaled, "X_features_individual_cgm")
check_nans(X_features_individual_scaled, "X_features_individual_scaled")
check_nans(X_features_daily_scaled, "X_features_daily_scaled")
for key in y_labels:
    check_nans(y_labels[key], f"y_label {key}")

#del filtered_data_dict, X_features_daily_cgm, X_features_individual_cgm

There are infinite values in the data.
There are infinite values in the data.
NaNs in the Data ...
X_matrices_power: 0  NaNs with shape: (1682, 288, 288, 3)
X_features_daily_cgm: 0  NaNs with shape: (1682, 132)
X_features_individual_cgm: 0  NaNs with shape: (1682, 154)
X_features_individual_scaled: 159006  NaNs with shape: (1682, 171)
X_features_daily_scaled: 216935  NaNs with shape: (1682, 327)
y_label hypo_early_night: 0  NaNs with shape: (1682, 1)
y_label hypo_night: 0  NaNs with shape: (1682, 1)
y_label hypo_long_night: 0  NaNs with shape: (1682, 1)
y_label hypo_night_morning: 0  NaNs with shape: (1682, 1)
y_label hyper_day: 0  NaNs with shape: (1682, 1)
y_label hypo_late_night: 0  NaNs with shape: (1682, 1)
y_label hyper_night: 0  NaNs with shape: (1682, 1)
y_label hyper_early_night: 0  NaNs with shape: (1682, 1)
y_label hypo_morning: 0  NaNs with shape: (1682, 1)
y_label hypo_day: 0  NaNs with shape: (1682, 1)


In [6]:
filtered_data_dict = {}
for participant_id, days_data in data_dict.items():
    filtered_days_data = []
    for current_date, day_data in days_data.items():

        if (day_data['len_original_cgm'] > 228 and np.sum(day_data['CGM_stats_daily_original'].isna()).sum()<1):
            filtered_days_data.append(day_data)
    
    if filtered_days_data:
        filtered_data_dict[participant_id] = filtered_days_data

with open('/home/ma98/data_generated/pediatric_cgm_mex_morl_smooth.dill', 'wb') as f:
    dill.dump(filtered_data_dict, f)
    
print(len(data_dict),len(filtered_data_dict))

# Prepare the groups based on participant IDs
participant_ids = []
X_features_individual_cgm, X_features_daily_cgm = [], []
X_matrices_power = [] #, X_matrices_coeffs
y_labels = { 
    "hypo_early_night": [], "hypo_night": [], "hypo_long_night": [],
    "hypo_night_morning": [], "hyper_day": [], "hypo_late_night": [],
    "hyper_night": [], "hyper_early_night": [], "hypo_morning": [],
    "hypo_day": []
}

for participant_id, days_data in filtered_data_dict.items():
    participant_ids.extend([participant_id] * len(days_data))
    for day_data in days_data:
        X_matrices_power.append(day_data['full_matrix_cgm_power'])
        # X_matrices_coeffs.append(day_data['full_matrix_cgm_coeffs'])
        X_features_daily_cgm.append(day_data['CGM_stats_daily_original'])
        X_features_individual_cgm.append(np.concatenate([
            [[day_data['A1c']]],
            day_data['CGM_stats_participant']], axis=1))
    
        # Store labels in the dictionary
        for label in y_labels.keys():
            y_labels[label].append(day_data['next_day_cgm_labels_original'][label])
    
participant_ids = np.array(participant_ids)
X_matrices_power = np.array(X_matrices_power)
# X_matrices_coeffs = np.array(X_matrices_coeffs)
X_features_daily_cgm = np.array(X_features_daily_cgm)
X_features_individual_cgm = np.array(X_features_individual_cgm)

# Convert labels to NumPy arrays
for key in y_labels:
    y_labels[key] = np.array(y_labels[key], dtype=np.float32)

# Normalize the individual features
def standard_scale(data):
    scaler = StandardScaler()
    if np.isinf(data).any():
        print("There are infinite values in the data.")
        
        # Replace infinite values with NaN (or a large number)
        data[np.isinf(data)] = np.nan
    return scaler.fit_transform(data.reshape(-1, data.shape[-1]))
    
# Apply vectorized scaling
X_features_individual_cgm_scaled = standard_scale(X_features_individual_cgm)
X_features_daily_cgm_scaled = standard_scale(X_features_daily_cgm)

def check_nans(arr, name):
    print(f"{name}: {np.isnan(arr).sum()}  NaNs with shape: {arr.shape}")

print('NaNs in the Data ...')
check_nans(X_matrices_power, "X_matrices_power")
# check_nans(X_matrices_coeffs, "X_matrices_coeffs")
check_nans(X_features_daily_cgm_scaled, "X_features_daily_cgm")
check_nans(X_features_individual_cgm_scaled, "X_features_individual_cgm")
for key in y_labels:
    check_nans(y_labels[key], f"y_label {key}")

#del filtered_data_dict, X_features_daily_cgm, X_features_individual_cgm

231 230
NaNs in the Data ...
X_matrices_power: 0  NaNs with shape: (1704, 288, 288, 3)
X_features_daily_cgm: 0  NaNs with shape: (1704, 132)
X_features_individual_cgm: 0  NaNs with shape: (1704, 154)
y_label hypo_early_night: 0  NaNs with shape: (1704, 1)
y_label hypo_night: 0  NaNs with shape: (1704, 1)
y_label hypo_long_night: 0  NaNs with shape: (1704, 1)
y_label hypo_night_morning: 0  NaNs with shape: (1704, 1)
y_label hyper_day: 0  NaNs with shape: (1704, 1)
y_label hypo_late_night: 0  NaNs with shape: (1704, 1)
y_label hyper_night: 0  NaNs with shape: (1704, 1)
y_label hyper_early_night: 0  NaNs with shape: (1704, 1)
y_label hypo_morning: 0  NaNs with shape: (1704, 1)
y_label hypo_day: 0  NaNs with shape: (1704, 1)


In [7]:
def manual_standard_scaling(matrix):
    # mean = np.nanmean(matrix)
    return matrix / np.nanstd(matrix)
    
X_matrices_power_scaled = np.zeros_like(X_matrices_power)
for i in range(X_matrices_power.shape[0]):
    X_matrices_power_scaled[i, :, :, 0] = manual_standard_scaling(X_matrices_power[i, :, :, 0])
    X_matrices_power_scaled[i, :, :, 1] = manual_standard_scaling(X_matrices_power[i, :, :, 1])
    X_matrices_power_scaled[i, :, :, 2] = manual_standard_scaling(X_matrices_power[i, :, :, 2])

del X_matrices_power
import gc
gc.collect()


0

In [16]:
# Step 1: Save dataset as a memory-mapped file (Only do this once)
def save_memmap_data(X, filename_prefix):
    if not os.path.exists(filename_prefix):
        shape = X.shape
        dtype = X.dtype
        print(dtype)
        memmap_data = np.memmap(f"{filename_prefix}.npy", dtype=dtype, mode='w+', shape=shape)
        memmap_data[:] = X[:]  # Copy data into the memory-mapped array
        del memmap_data  # Explicitly close the memory-mapped file

save_memmap_data(X_features_daily_cgm_scaled, "/home/__/data_generated/scaled_pediatric_features_individual")

memmap_file = "/home/__/data_generated/scaled_pediatric_features_individual.npy"
shape =(1682, 171)
dtype = X_features_individual_scaled.dtype
matrices_memmap = np.memmap(memmap_file, mode='r',dtype=dtype, shape=shape)
print(matrices_memmap.shape)
print(matrices_memmap.dtype)
check_nans(matrices_memmap, "X_features_individual_scaled")



save_memmap_data(X_features_individual, "/home/__/data_generated/pediatric_features_individual")

memmap_file = "/home/__/data_generated/scaled_pediatric_features_individual.npy"
shape = (1682, 171)
dtype = X_features_individual_cgm_scaled.dtype
matrices_memmap = np.memmap(memmap_file, mode='r',dtype=dtype, shape=shape)
print(matrices_memmap.shape)
print(matrices_memmap.dtype)
check_nans(matrices_memmap, "X_features_individual")



float64
(1682, 171)
float64
X_features_individual_scaled: 159006  NaNs with shape: (1682, 171)
float64
(1682, 171)
float64
X_features_individual: 159006  NaNs with shape: (1682, 171)


In [12]:

# Step 1: Save dataset as a memory-mapped file (Only do this once)
def save_memmap_data(X, filename_prefix):
    if not os.path.exists(filename_prefix):
        shape = X.shape
        dtype = X.dtype
        memmap_data = np.memmap(f"{filename_prefix}.npy", dtype=dtype, mode='w+', shape=shape)
        memmap_data[:] = X[:]  # Copy data into the memory-mapped array
        del memmap_data  # Explicitly close the memory-mapped file

save_memmap_data(X_matrices_power_scaled, "/home/__/data_generated/pediatric_cgm_matrices")

memmap_file = "/home/__/data_generated/pediatric_cgm_matrices.npy"
shape = (1704, 288, 288, 3)
matrices_memmap = np.memmap(memmap_file, dtype=np.float64, mode='r', shape=shape)
print(matrices_memmap.shape)
print(matrices_memmap.dtype)

def save_memmap_data(X, filename_prefix):
    if not os.path.exists(filename_prefix):
        shape = X.shape
        # dtype = X.dtype
        memmap_data = np.memmap(f"{filename_prefix}.npy", dtype=np.float16, mode='w+', shape=shape)
        memmap_data[:] = X[:]  # Copy data into the memory-mapped array
        del memmap_data  # Explicitly close the memory-mapped file
save_memmap_data(participant_ids, "/home/__/data_generated/pediatric_cgm_participant_ids")

participant_ids_memmap = np.memmap("/home/__/data_generated/pediatric_cgm_participant_ids.npy", dtype=np.float16, mode='r')
print(participant_ids_memmap.shape)
print(participant_ids_memmap.dtype)


# Path to store memory-mapped dataset
memmap_file = "/home/__/data_generated/pediatric_cgm_labels.npy"
dtype = np.float16
shape = (1704, 1)
# if not os.path.exists(memmap_file):
labels_memmap = np.memmap(memmap_file, dtype=dtype, mode='w+', shape=shape)
labels_memmap[:] = y_labels['hypo_night'][:]  # Copy data
del labels_memmap  # Flush changes to disk

# Step 2: Load memory-mapped dataset instead of keeping it in RAM
labels_memmap = np.memmap(memmap_file, dtype=np.float16, mode='r')
print(labels_memmap.shape)
print(labels_memmap.dtype)


(1704, 288, 288, 3)
float64
(1704,)
float16
(1704,)
float16
